# Season 19 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [1]:
import pandas
%matplotlib inline
from blaseball_mike.models import *
from blaseball_mike.tables import StatType
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown
from copy import deepcopy

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

# Fix Attractors
real_pies = deepcopy(pies)
for p in real_pies.lineup + real_pies.rotation:
    p.batting_rating = None
    p.pitching_rating = None
    p.baserunning_rating = None
    p.defense_rating = None

In [2]:
sim = SimulationData.load()
display(Markdown(f"**Last Updated Season {sim.season}, Day {sim.day}**"))

**Last Updated Season 19, Day 102**

---
## Decrees

Decrees are chosen by majority votes across all teams. `1` decrees will pass this season.

### Repeal Turntables
**Repeal the Turntables Rule.**

Repeals the Turntables Rule.

### Under Achiever

**unwin the underbracket. crown the underchampions.**


In [3]:
display(HTMLWrapper("<span style=\"font-family: 'Lora', 'Courier New', monospace, serif;font-weight: 700;color: #a16dc3;text-shadow:0 0 10px #a16dc3;\">as above, so below</span>"))

___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Equivalent Exchange
**Swap a Player from a Team in the opposite Subleague with a Player from your Team. They must be within 2 combined stars of each other.**

See below for the 20 best swap candidates for a particular player, sorted by highest stats in the applicable position.

In [ ]:
wild = Subleague.load("aabc11a1-81af-4036-9f18-229c759ca8a9")
all_wild = []
for team in wild.teams.values():
    all_wild.extend(team.lineup + team.rotation)

In [ ]:
all_total_stars = get_total_stars(all_wild)
for player in pies.lineup:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.batting_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_batting_stlats([player] + player_matches[0:20]), maxVal=1.5))

In [ ]:
for player in pies.rotation:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.pitching_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20], include_team=True))
    display(set_heatmap(get_pitching_stlats([player] + player_matches[0:20]), maxVal=1.5))

### Foreshadow
**Swap a Player from your Shadows to your active roster.**

There are 14 shadows players to swap with our 12 existing players. Below is a breakdown comparing the best shadows
players with the worst player in the particular position.

Note that position does not matter for the swap: any shadows player can bat or pitch.


**Batting:**

In [ ]:
shadows = real_pies.bullpen + real_pies.bench
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

display(get_stars([worst_batter] + best_batting_shadows))
display(set_heatmap(get_batting_stlats([worst_batter] + best_batting_shadows), maxVal=1))

**Pitching:**

In [ ]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

display(get_stars([worst_pitcher] + best_pitching_shadows))
display(set_heatmap(get_pitching_stlats([worst_pitcher] + best_pitching_shadows), maxVal=1))

### Move
**Move a Player on your Team to the desired location on your Roster.**

Move would move a player from any location on the team (including Shadows) to another position, increasing the number
of players in that position. See below for moving our best hitting pitcher or best pitching hitter into their preferred
position.

**Batting**

In [ ]:
bhp = best_hitting_pitcher(real_pies)
bph = best_pitching_hitter(real_pies)

display(Markdown(f"Moving {bhp.name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, bhp, "lineup")))

display(Markdown(f"Moving {worst_batter.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_batter, "bench")))

**Pitching**

In [ ]:
display(Markdown(f"Moving {bph.name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, bph, "rotation")))

display(Markdown(f"Moving {worst_pitcher.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_pitcher, "bullpen")))

### Item Move
**Move an Item from your Team to a Player on your Team, repairing it fully.**

See below for a list of all non-base items available to swap. Items can be moved to any player, regardless of if they
have an item or not. Items that break or are dropped will likely still be applicable if the will vote is submitted prior
to the item going away.

In [ ]:
all_items = []
for p in pies.lineup + pies.rotation + pies.bench + pies.bullpen:
    for item in p.items:
        if len(item.adjustments) > 1:
            all_items.append((p, item))

table = []
for p, i in all_items:
    mods = [Modification.load_one(a["mod"]).title for a in i.adjustments if a["type"] == 0]
    table.append(pandas.Series({"Wielder": p.name, "Max Durability": i.durability, "Modifications": mods}, name=i.name))
pandas.DataFrame(table)


### Shadow Infuse
**Boost a Player in your Team's Shadows by 15% to 30% in a Stat Category of your choice.**

See below for how this would affect various team members at the 2 extreme percentages. This will only affect one player.

**15% Change:**

In [ ]:
infuse_batting_table, _, _ = improve_team_batting_table(real_pies, 0.15, shadows=True)
display(infuse_batting_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.15, shadows=True)), maxVal=1))

In [ ]:
infuse_pitching_table, _, _ = improve_team_pitching_table(real_pies, 0.15, shadows=True)
display(infuse_pitching_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.15, shadows=True)), maxVal=1))


**30% Change:**

In [ ]:
infuse_lineup_table, _, _ = improve_team_batting_table(real_pies, 0.30, shadows=True)
display(infuse_lineup_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.30, shadows=True)), maxVal=1))

In [ ]:
infuse_rotation_table, _, _ = improve_team_pitching_table(real_pies, 0.30, shadows=True)
display(infuse_rotation_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.30, shadows=True)), maxVal=1))


### Shadow Revoke
**Revoke a Player from your Team's Shadows.**

This would remove a player from our shadows by moving them to another team, without providing a replacement.

### Reform
**Re-roll an eligible Permanent Modification for a Player on your Team.**

Below is a list of all players with modifications that could be rerolled. It would be better to reroll modifications
with negative effects.

In [ ]:
mod_players = [x for x in pies.lineup + pies.rotation if len(x.perm_attr) > 0]
mod_list = pandas.DataFrame()
for player in mod_players:
    data = pandas.Series({"mods": [x.title for x in player.perm_attr if x.id not in UNREFORMED_MODS]}, name=player.name)
    if len(data["mods"]) == 0:
        continue
    mod_list = mod_list.append(data)
mod_list

### Alternate Trust
**Call in an Alternate for a Player on your Team, with a mystery Benefit.**

An Alternate is a player reroll and the Benefit is an unknown modification. Alternates were changed for S18 and onward
to always result in a player with the same combined stars. This would be useful for rerolling a player that has a
less-optimal player attribute distribution.

___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### Now Hiring
**Recruit a Random Player with the Roamin' Modification to your Team.**

See below for list of all active players with Roamin'. This will not remove an existing player from our team.

In [ ]:
league = League.load()
divisions = league.subleagues
all_teams = []
for division in divisions.values():
    all_teams.extend(list(division.teams.values()))

In [ ]:
siphons = []
for team in all_teams:
    siphons.extend([x for x in team.lineup + team.rotation if "WANDERER" in x._perm_attr_ids])
get_stars(siphons, include_team=True)

### Super Roamin' Fifth Base
**Give the best Baserunner in your Team's Lineup the Legendary Super Roamin' Fifth Base.**

Unknown, likely an Item with both Super Roamin' and Fifth Base. A Super Roamin' player will likely immediately leave
our team.

In [ ]:
p = pies.lineup.copy()
p.sort(key=lambda x: x.baserunning_rating, reverse=True)
display(Markdown(f"Will be given to {p[0].name}."))

### Fax Preparation
**Move the Best Pitcher in your Team's Lineup to your Shadows.**

See below for analysis.

In [ ]:
bph = best_pitching_hitter(pies)
display(Markdown(f"**Moving {bph.name} to the Shadows**"))

pandas.DataFrame(remove_player(pies, bph))

### Hits Keep Coming
**Sort your Lineup in order of Tragicness.**

See below for new lineup order.

In [ ]:
new_lineup = pies.lineup.copy()
new_lineup.sort(key=lambda x: x.tragicness, reverse=True)
get_stars(new_lineup)

### Outdoorsy
**A random Player on your Team will play better in Grandiose Ballparks and worse in Ungrandiose Ballparks.**

This will likely cause a player to perform better in Stadiums with Grandiosity >0.5 and worst in Stadiums with
Grandiosity <0.5.

In [ ]:
stadiums = Stadium.load_all()
table = pandas.DataFrame()
for s in stadiums.values():
    table = table.append(pandas.Series({"Grandiosity": s.grandiosity}, name=s.team_id.nickname))
table = table.sort_values(by="Grandiosity")
set_heatmap(table, maxVal=1)

### Gaudy
**A random Player on your Team will play better in Ballparks with more Modifications.**

See below for list of Stadiums and their number of modifications.

In [ ]:
table = pandas.DataFrame()
for s in stadiums.values():
    table = table.append(pandas.Series({"Number of Mods": len(s.mods)}, name=s.team_id.nickname))
table.sort_values(by="Number of Mods")

### Cluttered
**A random Player on your Team will play better in Filthier Ballparks.**

See below for list of Filthiest Stadiums.

In [ ]:
table = pandas.DataFrame()
for s in stadiums.values():
    table = table.append(pandas.Series({"Filthiness": s.filthiness}, name=s.team_id.nickname))
table = table.sort_values(by="Filthiness")
set_heatmap(table, maxVal=1)

### Guarded
**A random Player on your Team will play better in Fortified Ballparks and worse in Unfortified Ballparks.**

This will likely cause a player to perform better in Stadiums with Fortification >0.5 and worst in Stadiums with
Fortification <0.5.

In [ ]:
table = pandas.DataFrame()
for s in stadiums.values():
    table = table.append(pandas.Series({"Fortification": s.fortification}, name=s.team_id.nickname))
table = table.sort_values(by="Fortification")
set_heatmap(table, maxVal=1)

### Practice
**Your Team's Lineup and Rotation will have their Defense Boosted from 1% to 4%.**

**Your Team's Lineup will have their Baserunning Boosted from 1% to 4%.**

**Your Team's Rotation will have their Pitching Boosted from 1% to 4%.**

**Your Team's Lineup will have their Hitting Boosted from 1% to 4%.**

These blessings increase the team-wide stats. See below for worst and best case rolls.

#### Batting Practice

**Worst Case**

In [ ]:
table, total, avg = improve_team_batting_table(real_pies, 0.01)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.01)), maxVal=1))

**Best Case**

In [ ]:
table, total, avg = improve_team_batting_table(real_pies, 0.04)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.04)), maxVal=1))


#### Pitching Practice

**Worst Case**

In [ ]:
table, total, avg = improve_team_pitching_table(real_pies, 0.01)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.01)), maxVal=1))

**Best Case**

In [ ]:
table, total, avg = improve_team_pitching_table(real_pies, 0.04)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.04)), maxVal=1))

#### Running Practice

**Worst Case**

In [ ]:
table, total, avg = improve_team_baserunning_table(real_pies, 0.01)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.01)), maxVal=1))

**Best Case**

In [ ]:
table, total, avg = improve_team_baserunning_table(real_pies, 0.04)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.04)), maxVal=1))

#### Defense Practice

**Worst Case**

In [ ]:
table, total, avg = improve_team_defense_table(real_pies, 0.01)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.01)), maxVal=1))

**Best Case**

In [ ]:
table, total, avg = improve_team_defense_table(real_pies, 0.04)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.04)), maxVal=1))

### Shadow Boosts
**Your Team's Shadows will have their Hitting Boosted from -5% to 20%.**

**Your Team's Shadows will have their Pitching Boosted from -5% to 20%.**

**Your Team's Shadows will have their Baserunning Boosted from -5% to 20%.**

**Your Team's Shadows will have their Defense Boosted from -5% to 20%.**

These blessings increase the team-wide stats or our shadow players. See below for worst and best case rolls. This
assumes that it will only boost the respective shadow positions, so batting will only affect the bench, etc.

#### Shadow of the Bats

**Worst Case**

In [ ]:
table, total, avg = improve_team_batting_table(real_pies, -0.05, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, -0.05, shadows=True)), maxVal=1))

**Best Case**

In [ ]:
table, total, avg = improve_team_batting_table(real_pies, 0.20, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_batting_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_batting_stars"])))
display(table)
display(set_heatmap(get_batting_stlats(improve_team_batting(real_pies, 0.20, shadows=True)), maxVal=1))


#### Fax Numbers

**Worst Case**

In [ ]:
table, total, avg = improve_team_pitching_table(real_pies, -0.05, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, -0.05, shadows=True)), maxVal=1))

**Best Case**

In [ ]:
table, total, avg = improve_team_pitching_table(real_pies, 0.20, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_pitching_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_pitching_stars"])))
display(table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(real_pies, 0.20, shadows=True)), maxVal=1))

#### Shadowrun

**Worst Case**

In [ ]:
table, total, avg = improve_team_baserunning_table(real_pies, -0.05, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, -0.05, shadows=True)), maxVal=1))

**Best Case**

In [ ]:
table, total, avg = improve_team_baserunning_table(real_pies, 0.20, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_baserunning_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_baserunning_stars"])))
display(table)
display(set_heatmap(get_baserunning_stlats(improve_team_baserunning(real_pies, 0.20, shadows=True)), maxVal=1))

#### Cover Letters

**Worst Case**

In [ ]:
table, total, avg = improve_team_defense_table(real_pies, -0.05, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, -0.05, shadows=True)), maxVal=1))

**Best Case**

In [ ]:
table, total, avg = improve_team_defense_table(real_pies, 0.20, shadows=True)
display(Markdown("Change in Total stars: " + str(total["change_in_defense_stars"])))
display(Markdown("Change in Average stars: " + str(avg["change_in_defense_stars"])))
display(table)
display(set_heatmap(get_defense_stlats(improve_team_defense(real_pies, 0.20, shadows=True)), maxVal=1))

### Junk Baller
**A Random Player in your Team's Rotation will have their Unthwackability Boosted from -20% to +60%.**

Unthwackability is the player attribute most associated with increasing ground and flyouts and reducing overall hits.
See below for attribute change for best and worst case rolls.

**Worst Case**

In [ ]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"unthwackability": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

**Best Case**

In [ ]:
new_team = []
for player in real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"unthwackability": 0.6}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

### Walk it Off
**A Random Player in your Team's Lineup will have their Moxie Boosted from -20% to +60%.**

Moxie is the player attribute most associated with plate discipline, and increases walks.
See below for attribute change for best and worst case rolls.

**Worst Case**

In [ ]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"moxie": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

**Best Case**

In [ ]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"moxie": 0.6}))
display(get_stars(new_team))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

### Feast or Famine
**A Random Player from your Team's Lineup will have their Indulgence Boosted from -20% to +60%.**

Indulgence is the player attribute most associated with baserunners getting advancement on batter outs.
See below for attribute change for best and worst case rolls.

**Worst Case**

In [ ]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"indulgence": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_baserunning_stlats(new_team), maxVal=1))

**Best Case**

In [ ]:
new_team = []
for player in real_pies.lineup:
    new_team.append(player.simulated_copy(buffs={"indulgence": 0.6}))
display(get_stars(new_team))
display(set_heatmap(get_baserunning_stlats(new_team), maxVal=1))

### Revelations
**A Random Player on your Team will have their Omniscience Boosted from -20% to +60%.**

Omniscience is the player attribute most associated with fielding outs and general defense.
See below for attribute change for best and worst case rolls.

**Worst Case**

In [ ]:
new_team = []
for player in real_pies.lineup + real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"omniscience": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_defense_stlats(new_team), maxVal=1))

**Best Case**

In [ ]:
new_team = []
for player in real_pies.lineup + real_pies.rotation:
    new_team.append(player.simulated_copy(buffs={"omniscience": 0.6}))
display(get_stars(new_team))
display(set_heatmap(get_defense_stlats(new_team), maxVal=1))

### Supply Runs
**Each Player in your Team's Shadows with an available Item Slot will open a Generic Sunglasses Crate.**

**Each Player in your Team's Shadows with an available Item Slot will open a Generic Helmet Crate.**


For reference, these items boost the following:


| Item Type  | Attributes | Stat |
| ---------- | ---------- | ---- |
| Sunglasses | +Cinnamon | Vibes |
| Helmet     | +Pressurization | Vibes |


This would provide Generic Sunglasses or Helmets to the following players:

In [ ]:
free_items = [x for x in pies.bench + pies.bullpen if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items)

### Targeted Evolution
**A random pitcher and a random hitter on your Team will Advance.**

An Advanced player will gain one evolution level, increasing their minimum star level by one for all stats and raising
their stars to the minimum if they are lower than that. It also allows players to hold an additional item.

### Headliners
**Arrange your lineup in order of their Idolatry.**

The exact ordering of our team by Idolatry is unknown. With sufficient team organization this could be used to improve
lineup efficiency.


### Moderation

**Your Team will accept Unruns after each non-loss so that they only non-lose by 1 Run.**

This will reduce our score in games where we Win to the opponent's score plus 1. So if the Pies won a game 7-3, it would
reduce the score to 4-3. This would be helpful for lowering eDensity, as Runs are included in that equation.

### Underhanded

**Make contact, I dare you. Make a random Player in your Team's Rotation Underhanded. Home Runs will be Unruns against them.**

This would cause opponents to earn negative runs when they hit a Home Run against the pitcher with this Modification.

### Subtractor

**A random Player in another Team's Lineup will become a Subtractor.**

The effect of Subtractor is unknown. Interestingly, this would affect another team.

### Go Big? Go Home

**A random Player in another Team's Lineup will become an Underachiever.**

The effect of Underachiever is unknown. Interestingly, this would affect another team.

## Gifts

Gifts are awarded to teams based on contributions from other teams. Each team creates a Wishlist of items that they
would like, and the top items on the wishlist are awarded during the Latesiesta.

### Collector's Editions

Collectors Editions move a Legendary player to the team. Replicas act as copies of that player and will turn to Dust
after elections.

Special Notes:
* Nagomi has fake Attractor stars, this analysis uses the player attributes.
* Chorby is still unstable, and is likely to get incinerated if possible.
* Chorby replicas have 0 soul, so are safe from consumers

In [ ]:
york = Player.find_by_name("York Silk")
nagomi = Player.find_by_name("Nagomi Mcdaniel")
nagomi.batting_rating = None
nagomi.baserunning_rating = None
nagomi.defense_rating = None
aldon = Player.find_by_name("Aldon Cashmoney")
goodwin = Player.find_by_name("Goodwin Morin")

pm = Player.find_by_name("Pitching Machine")
chorby = Player.find_by_name("Chorby Soul")

display(Markdown("#### Batting"))
display(set_heatmap(get_batting_stlats([york, nagomi, aldon, goodwin] + pies.lineup), maxVal=1.5))

display(Markdown("Adding York Silk"))
display(pandas.DataFrame(move_player(pies, york, position='lineup')))

display(Markdown("Adding Nagomi Mcdaniel"))
display(pandas.DataFrame(move_player(pies, nagomi, position='lineup')))

display(Markdown("Adding Aldon Cashmoney"))
display(pandas.DataFrame(move_player(pies, aldon, position='lineup')))

display(Markdown("Adding Goodwin Morin"))
display(pandas.DataFrame(move_player(pies, goodwin, position='lineup')))

display(Markdown("#### Pitching"))
display(set_heatmap(get_pitching_stlats([pm, chorby] + pies.rotation), maxVal=1.5))

display(Markdown("Adding Pitching Machine"))
display(pandas.DataFrame(move_player(pies, pm, position='rotation')))

display(Markdown("Adding Chorby Soul"))
display(pandas.DataFrame(move_player(pies, chorby, position='rotation')))

### Solo Editions

These gifts provide a random player with a benefit for the lateseason and postseason.

* Fourth Strike: Player will be out on 4 strikes instead of 3.
* Walk in the Park: Player will earn a walk on 2 balls rather than 3.
* Ambitious: Player will overperform in the postseason.

### Team Editions

These gifts provide a team-wide benefit for the lateseason and postseason.

* Life of the Party: Increases the improvement of each party while in Party Time.
* Late to the Party: Team will overperform in the Lateseason.
* Home Field Advantage: Team will start each home game with 1 run. Playoff games alternate between home and away each game.
* Fireproof: Team will be safe from incinerations. Pies only have 1 Solar Eclipse game in the lateseason.
* Soundproof: Team will be safe from Feedback. Pies have 2 Feedback weather games in the lateseason.
* Gravity: Team will be safe from Reverb. Pies only have 1 Reverb weather game in the lateseason.

### Handcrafted Drops

These will provide Items which have additional stat increases and random special effects.

|Type | Attribute | Stat |
| --- | --- | --- |
| Cannon | +Overpowerment | Pitching |
| Jersey | +Musclitude | Batting |
| Necklace | +Moxie | Batting |
| Phone | ??? | ??? |
| Quill | ??? | ??? |
| Ring | +Ruthlessness | Pitching |
| Sunglasses | +Cinnamon | Vibes |

See below for a list of players that can receive them.

**Jersey, Necklace**

In [ ]:
free_items_lineup = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_lineup)

**Cannon, Quill, Ring**

In [ ]:
free_items_rotation= [x for x in pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_rotation)

**Phone, Sunglasses**

In [ ]:
free_items = [x for x in pies.lineup + pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items)

### Soul Patches

**Back by Popular Demand! Give 1 Soul to your least Soul Full player. Repeat 10 times.**

See below for final Soul totals. Low soul is dangerous, as Players will become Redacted if their soul hits 0 and
Consumers will drain 1 soul per chomp.

In [ ]:
players = deepcopy(pies.rotation) + deepcopy(pies.lineup)
for i in range(0, 10):
    players.sort(key=lambda x: x.soul)
    worst = players[0]
    worst.soul += 1
    players = [x for x in players if x.id != worst.id] + [worst]
players.sort(key=lambda x: x.soul)
x = pandas.DataFrame()
for p in players:
    old = [x for x in pies.rotation + pies.lineup if x.id == p.id][0]
    x = x.append(pandas.Series({"Soul Added": p.soul - old.soul,"Resulting Soul": p.soul}, name=p.name))
x

### Bargain Bin

**Give a random Player on your Team the Best Item from the Bargain Bin!**

The Bargain Bin will retrieve one random previously dropped item. Below is a list of all non-base items not currently
held by players.

In [ ]:
all_items = Item.load_all()
rare_items = [x for x in all_items.values() if len(x.adjustments) > 1]
players = Player.load_all()
carried_items = []
for p in players.values():
    carried_items.extend(p.items)
dropped_items = [x for x in rare_items if x not in carried_items]
dropped_items.sort(key=lambda x: len(x.adjustments), reverse=True)

table = []
for i in dropped_items:
    mods = [Modification.load_one(a["mod"]).title for a in i.adjustments if a["type"] == 0]
    table.append(pandas.Series({"Durability": i.durability, "Modifications": mods}, name=i.name))
pandas.DataFrame(table)

---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
